# Lex Guardian Notebook
In this notebook, we experiment on the implementation of the law guardian

## Import the Necessary files and Libraries

In [2]:
import os

os.chdir(r"C:\D\New Desktop\Projects\Application\LexGuardian")
print(os.getcwd())


C:\D\New Desktop\Projects\Application\LexGuardian


In [3]:
!pip install HuggingFaceEndpoint

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement HuggingFaceEndpoint (from versions: none)
ERROR: No matching distribution found for HuggingFaceEndpoint

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Import libraries
from dotenv import load_dotenv
from lex_guardian.utils import load_config
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [5]:
# Load config and dotenv
config = load_config()
print(config)
load_dotenv()

{'data': {'data_path': 'data/raw/jbc.pdf'}, 'vectorstore': {'url': 'https://e5efed46-432a-4f52-8326-4eca09694ae0.us-east4-0.gcp.cloud.qdrant.io:6333'}}


True

## Process the documents

In [6]:
# Load the data

## Instantiate loader
loader = PyPDFLoader(config["data"]["data_path"])
print(loader)
## load the data
documents = loader.load()
print(documents)
## Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
print(text_splitter)
text_chunks = text_splitter.split_documents(documents)
print(text_chunks)

[Document(metadata={'source': 'data/raw/jbc.pdf', 'page': 0}, page_content='1/47  \n \n \n \n \n \n \nAGREEMENT AND \nCONDITIONS OF \nCONTRACT FOR \nBUILDING WORKS  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nPUBLISHED BY  \nTHE JOINT BUILDING COUNCIL, KENYA  \nWITH THE SANCTION OF  \n• THE ARCHITECTURAL ASSOCIATION OF KENYA  \n• THE KENYA  ASSOCIATION OF BUILDING AND CIVIL ENGINEERING  CONTRACTORS  \n \nAPRIL 1999 EDITION  '), Document(metadata={'source': 'data/raw/jbc.pdf', 'page': 1}, page_content="2/47 1.0 DEFINITIONS  \n1.1 Architects instructions - Means instructions issued by the Architect to the Contractor in accordance \nwith these conditions and includes instructions issued by other agents of the Employer appointed unde r \narticle 2.13 of the agreement.  \n1.2    Bill s of quantities or contra ct bills - Means the document drawn up by the Quantity Surveyor and \npriced by the Contractor to arrive at the contract price.  \n1.3    Certificate o

In [7]:
## Instantialize vectore store and load documents
url = config["vectorstore"]["url"]
api_key = os.getenv("QDRANT_API_KEY")
print(api_key)
model_name = "BAAI/bge-base-en-v1.5"
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
)
vectorstore = Qdrant.from_documents(
    text_chunks,
    embeddings,
    url=url,
    prefer_grpc=False,
    api_key=api_key,
    collection_name="kpc",
)

-L_8Cap1CZyHU9gr7w9H0Lm1-nfMR5T7Eo-SHqjorz43DUSapa2sYA


C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-08-22 11:10:22,826 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2024-08-22 11:10:22,829 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransfor

## Retriever Initialization

In [8]:
from qdrant_client import QdrantClient

client = QdrantClient(url=url, api_key=api_key)

# Try a simple operation to test the connection
try:
    collections = client.get_collections()
    print(f"Available collections: {collections}")
except Exception as e:
    print(f"Error connecting to Qdrant: {e}")

2024-08-22 11:21:18,993 - httpx - INFO - HTTP Request: GET https://e5efed46-432a-4f52-8326-4eca09694ae0.us-east4-0.gcp.cloud.qdrant.io:6333/collections "HTTP/1.1 200 OK"


Available collections: collections=[CollectionDescription(name='constitution'), CollectionDescription(name='kpc')]


In [9]:
# Instantiate retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
print(retriever)

tags=['Qdrant', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x000001CF13050230> search_kwargs={'k': 5}


In [10]:
docs = retriever.invoke("What would be acceptable grounds for contract termination")
print(docs)

2024-08-22 11:21:19,877 - httpx - INFO - HTTP Request: POST https://e5efed46-432a-4f52-8326-4eca09694ae0.us-east4-0.gcp.cloud.qdrant.io:6333/collections/kpc/points/search "HTTP/1.1 200 OK"


[Document(metadata={'page': 38, 'source': 'data/raw/jbc.pdf', '_id': '066b29a2-3f7f-4405-933c-cd599d10d773', '_collection_name': 'kpc'}, page_content='delivery, terminate the contract, provided that such  notice shall not be given unreasonably or \nvexatiously. Upon such termination, the Contractor shall be paid by the Employer in respect \nof the work executed before such termination in accordance with the provisions of clause \n39.5.1 to 39.5.5.'), Document(metadata={'page': 38, 'source': 'data/raw/jbc.pdf', '_id': '005002ba-4bd8-4ad0-a96e-a12b19ba41b6', '_collection_name': 'kpc'}, page_content='contract may be terminated by either party giving to the other fourteen days notice by \nregistered post or recorded delivery. Upon termination the Contractor shall be paid by the \nEmployer in respect of the work c arried out before such termin ation in accordance with the \nprovisions of sub -clauses 39.5.1 t o 39.5.5 of these conditions.  \n40.2   In the event of the  Works or any part t h

In [11]:
# Get prompt
from langchain.prompts import PromptTemplate

prompt = """
You are an experienced legal advisor analyzing a contract. Your task is to provide accurate and helpful information based on the contract text provided. 
Please answer the question using only the information given in the context. If the answer cannot be found in the context, state that you don't have enough information to answer.

Contract text:
{context}

Question: {question}

Legal analysis:
"""

prompt_template = PromptTemplate.from_template(prompt)

## Create QA Chain

In [12]:
for i, doc in enumerate(docs, 1):
    print(f"Document {i}:")
    print(doc.page_content)
    print("-" * 50)
    

Document 1:
delivery, terminate the contract, provided that such  notice shall not be given unreasonably or 
vexatiously. Upon such termination, the Contractor shall be paid by the Employer in respect 
of the work executed before such termination in accordance with the provisions of clause 
39.5.1 to 39.5.5.
--------------------------------------------------
Document 2:
contract may be terminated by either party giving to the other fourteen days notice by 
registered post or recorded delivery. Upon termination the Contractor shall be paid by the 
Employer in respect of the work c arried out before such termin ation in accordance with the 
provisions of sub -clauses 39.5.1 t o 39.5.5 of these conditions.  
40.2   In the event of the  Works or any part t hereof or any goods or materials brought on the site for
--------------------------------------------------
Document 3:
NOW the condition of the above written bond is such that if the said Employer his executors, 
administrators, success

In [13]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [14]:
# LLM

repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.3,
)

C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\USER\.cache\huggingface\token
Login successful


In [15]:
# Chain using LangChain Expression Language
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)


In [16]:
def stream_response(query):
    for chunk in rag_chain.stream(query):
        print(chunk, end="", flush=True)

In [17]:
stream_response("clause 8.4")

2024-08-22 11:21:26,133 - httpx - INFO - HTTP Request: POST https://e5efed46-432a-4f52-8326-4eca09694ae0.us-east4-0.gcp.cloud.qdrant.io:6333/collections/kpc/points/search "HTTP/1.1 200 OK"


Clause 8.4 of the contract states that any error in description or quantity, or any omission of items from the contract bills or specifications, shall not invalidate the contract. Instead, these errors or omissions shall be corrected and deemed to be a variation required by the Architect. This means that even if there are discrepancies in the contract documents, the contract remains valid and the errors will be addressed as variations to the original contract. The contract price will not be adjusted or altered unless specified in the express provisions of the conditions. This clause is designed to ensure that minor errors or discrepancies do not lead to the termination of the contract, but are instead corrected and dealt with as variations.</s>

In [18]:
from lex_guardian.rag import instantiate_db, setup_retriever, setup_llm, setup_rag_chain

In [19]:
db = instantiate_db(config)
retriever = setup_retriever(db)

2024-08-22 11:21:38,919 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2024-08-22 11:21:38,922 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  warn_deprecated(


In [20]:
retriever.get_relevant_documents("clause 8.4")

C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
2024-08-22 11:21:55,795 - httpx - INFO - HTTP Request: POST https://e5efed46-432a-4f52-8326-4eca09694ae0.us-east4-0.gcp.cloud.qdrant.io:6333/collections/constitution/points/search "HTTP/1.1 200 OK"


[Document(metadata={'page': 24, 'source': 'data/raw/jbc.pdf', '_id': '30b3601a-8130-4d1a-9f24-7eff167882ab', '_collection_name': 'constitution'}, page_content='these conditions, it shall be deemed for the purposes of this sub -clause to have been included in \nthe contract bills.'),
 Document(metadata={'page': 9, 'source': 'data/raw/jbc.pdf', '_id': '8454c27c-6c0b-47c6-8653-03fd0e1086d5', '_collection_name': 'constitution'}, page_content='8.4    Any e rror in description or in quan tity or any omission of items from the contract bills or \nspecifications shall not vitiate this contract but shall be corrected and deemed to be a variation \nrequired by the Architect.  \n8.5    The contract  price shall not be adjusted or altered in any way wha tsoever otherwise than in \naccordance with the express provisions of these conditions.  \n8.6    Subject to sub -clause 8.4 of this condition, any error whether arithmetical or not in the'),
 Document(metadata={'page': 25, 'source': 'data/raw/jbc.

In [21]:
llm = setup_llm()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\USER\.cache\huggingface\token
Login successful


In [22]:
rag_chain = setup_rag_chain(llm=llm, retriever=retriever)

In [24]:
stream_response("my laws when I get arrested")

2024-08-22 11:21:57,656 - httpx - INFO - HTTP Request: POST https://e5efed46-432a-4f52-8326-4eca09694ae0.us-east4-0.gcp.cloud.qdrant.io:6333/collections/constitution/points/search "HTTP/1.1 200 OK"



    When you get arrested, there are certain laws that govern the process. However, the context provided seems to be about a construction contract between an Employer and a Contractor. It outlines the conditions under which the Employer can suspend or postpone the Works, and the Contractor's liability for loss or damage. 

    In this context, there's no direct information about laws when one gets arrested. Generally, in Kenya, when a person is arrested, they have the right to remain silent, and they should be informed of the reason for their arrest. They also have the right to legal representation and to be brought before a court within 24 hours. 

    If you're asking about the Contractor's liability in the construction contract, the Contractor is liable for loss or damage, except for such loss or damage that is at the risk of the Employer under certain clauses. The Contractor's equipment is also at the sole risk of the Employer regarding loss or damage by specific causes. 

    If 